In [53]:
import numpy as np
import pandas as pd

---
### 평점을 주지 않은 영화를 리스트로 객체로 반환하는 함수

In [54]:
def get_unseen_movies(rating_matrix, userId):
    # userId로 입력받은  사용자의 모든 영화 정보를 추출해 Series로 반환함
    # 반환된 user rating은 영화명(title)을 인덱스로 가지는 Series 객체
    user_rating = rating_matrix.loc[userId,:]
    
    # user_rating이 0보다 크면 기존에 관람한 영화임.
    # 대상 인덱스를 추출해 객체로 만듬
    already_seen = user_rating[user_rating > 0].index.tolist()
    
    # 모든 영화명을 list 객체로 만듦
    movies_list = rating_matrix.columns.tolist()
    
    # list comprehension으로 already_seen에 해당하는 영화는 movies_list에서 제외함
    unseen_list = [movie for movie in movies_list if movie not in already_seen]
    return unseen_list

---
### 영화 추천 함수

In [51]:
def recomm_movie_by_userid(ratings_pred_matrix, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id, 인덱스와 unseen_list로 들어온  영화명 칼럼을 추출해
    # 가장 예측 평점이 높은 순으로 정렬함.
    recomm_movies = ratings_pred_matrix.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    recomm_movies = pd.DataFrame(recomm_movies.values, index = recomm_movies.index,
                                columns = ['pred_score'])
    return recomm_movies

---
### RMSE

In [127]:
from sklearn.metrics import mean_squared_error

def get_rmse(R,P,Q,non_zeros):
    error = 0
    
    # 두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix=np.dot(P,Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스를 추출해 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_idx = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_idx = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_idx,y_non_zero_idx]
    
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_idx,y_non_zero_idx]
    
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

---
### matrix_factorization

In [2]:
def matrix_factorization(R, K, steps = 200, learning_rate=0.01, r_lambda=0.01):
#     R : 원본 행렬
#     K : 잠재요인의 수
#     learning_rae = 0.01 >> 학습률
#     r_lambda = 0.01 >> L2 규제
    num_users,num_items = R.shape
    # 이거는 사용자-아이템 기반
    
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))
    # P >> 사용자 잠재요인 행렬
    # Q >> 아이템 잠재요인 행렬 
    
    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장
    non_zeros = [(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0]
    
    # SGD 기법으로 P와 Q 매트릭스를 계속 업데이트 
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값을 구함
            eij = r-np.dot(P[i,:], Q[j,:].T)
            P[i,:] = P[i,:] + learning_rate*(eij*Q[j,:]-r_lambda*P[i,:]) 
            Q[j,:] = Q[j,:] + learning_rate*(eij*P[i,:]-r_lambda*Q[j,:])
            
        rmse = get_rmse(R,P,Q, non_zeros)
        if (step % 10) == 0:
            print('### iteration step : ', step, 'RMSE : ', rmse)
    return P, Q

In [4]:
movies = pd.read_csv('data/ml-latest/movies.csv')
ratings = pd.read_csv('data/ml-latest/ratings.csv')

In [5]:
ratings.drop(['timestamp'],axis=1,inplace=True)
ratings = ratings[['userId','movieId','rating']]
ratings.head()

,userId,movieId,rating
0,1,307,3.5
1,1,481,3.5
2,1,1091,1.5
3,1,1257,4.5
4,1,1449,4.5


In [131]:
ratings = ratings[:500000]
ratings.head()

,userId,movieId,rating
0,1,307,3.5
1,1,481,3.5
2,1,1091,1.5
3,1,1257,4.5
4,1,1449,4.5


In [7]:
rating_movies = pd.merge(ratings,movies, on='movieId')
rating_movies

,userId,movieId,rating,title,genres
0,1,307,3.5,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
1,6,307,4.0,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
2,56,307,4.0,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,71,307,5.0,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
4,84,307,3.0,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
...,...,...,...,...,...
499995,5069,80454,3.0,Princess (Prinsessa) (2010),Drama
499996,5069,80584,3.0,"Patrik Age 1.5 (Patrik 1,5) (2008)",Comedy|Drama|Romance
499997,5069,83760,1.5,V2: Dead Angel (Vares 2 - Jäätynyt Enkeli) (2007),Action|Comedy|Crime
499998,5073,439,3.0,Dangerous Game (1993),Drama


In [9]:
rating_matrix = rating_movies.pivot_table('rating',index='userId',columns='title')
rating_matrix

title,"""Great Performances"" Cats (1998)",#Horror (2015),$ (Dollars) (1971),$5 a Day (2008),'71 (2014),'Hellboy': The Seeds of Creation (2004),'R Xmas (2001),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),...,¡Three Amigos! (1986),À nos amours (1983),À nous la liberté (Freedom for Us) (1931),À propos de Nice (1930),İtirazım Var (2014),Он вам не Димон (2017),Оно (1990),Сердца трех (1992),Я худею (2018),チェブラーシカ (2010)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
P,Q = matrix_factorization(rating_matrix.values, K = 50, steps=200,learning_rate=0.01,
                           r_lambda=0.01)

### iteration step :  0 RMSE :  2.496955145647253
### iteration step :  10 RMSE :  0.7335461862330657
### iteration step :  20 RMSE :  0.5492336780312425
### iteration step :  30 RMSE :  0.46424385902982696
### iteration step :  40 RMSE :  0.4217829705939318
### iteration step :  50 RMSE :  0.3980113460537153
### iteration step :  60 RMSE :  0.3831515215842209
### iteration step :  70 RMSE :  0.37301118310755627
### iteration step :  80 RMSE :  0.3656464575581724
### iteration step :  90 RMSE :  0.360047010014072
### iteration step :  100 RMSE :  0.3556376678137477
### iteration step :  110 RMSE :  0.35206990850549347
### iteration step :  120 RMSE :  0.3491202176638753
### iteration step :  130 RMSE :  0.346638023963148
### iteration step :  140 RMSE :  0.3445179439397271
### iteration step :  150 RMSE :  0.3426839724942235
### iteration step :  160 RMSE :  0.34107992806976867
### iteration step :  170 RMSE :  0.3396634101925104
### iteration step :  180 RMSE :  0.3384018392518488
###

In [68]:
display(P) # 사용자 - 잠재요인 행렬
P.shape

array([[ 0.35599291, -0.10987868,  0.04626175, ...,  0.02966964,
         0.29034151,  0.27736949],
       [ 0.56807624,  0.02970512,  0.32705367, ..., -0.22020422,
         0.25444588,  0.05485764],
       [ 0.30041696,  0.02938043,  0.52666533, ..., -0.02544488,
         0.52381955,  0.24365333],
       ...,
       [ 0.25228827,  0.05478081,  0.55919929, ..., -0.08929458,
         0.36501249, -0.04724237],
       [ 0.28446732, -0.10347284,  0.39682981, ..., -0.3658256 ,
         0.29778756,  0.28432959],
       [ 0.06112744,  0.14273225,  0.04980332, ..., -0.43847934,
         0.56460235,  0.06767285]])

(5081, 50)

In [133]:
display(Q) # 아이템(영화제목) - 잠재요인 행렬
Q.shape

array([[ 0.31401422,  0.0377941 ,  0.33943434, ..., -0.08195066,
         0.16173246,  0.00734377],
       [ 0.02727246,  0.02549693,  0.0439645 , ..., -0.03981129,
         0.03410162,  0.00124451],
       [ 0.1689633 , -0.03571269, -0.13204279, ..., -0.1755713 ,
         0.07766028, -0.05956261],
       ...,
       [ 0.0021157 , -0.01219002,  0.01725528, ..., -0.00502689,
         0.03877886, -0.03949737],
       [ 0.25690733,  0.3027502 ,  0.14019826, ..., -0.22556376,
         0.14822396,  0.01774761],
       [ 0.10029644,  0.03053389,  0.15444709, ..., -0.13674602,
         0.15920207, -0.26948711]])

(17817, 50)

In [71]:
pred_matrix = np.dot(P,Q.T)
ratings_pred_matrix = pd.DataFrame(pred_matrix, columns = rating_matrix.columns,
                                  index = rating_matrix.index)
ratings_pred_matrix

title,"""Great Performances"" Cats (1998)",#Horror (2015),$ (Dollars) (1971),$5 a Day (2008),'71 (2014),'Hellboy': The Seeds of Creation (2004),'R Xmas (2001),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),...,¡Three Amigos! (1986),À nos amours (1983),À nous la liberté (Freedom for Us) (1931),À propos de Nice (1930),İtirazım Var (2014),Он вам не Димон (2017),Оно (1990),Сердца трех (1992),Я худею (2018),チェブラーシカ (2010)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.074564,0.559204,1.282402,1.286171,3.673177,3.424377,1.497756,4.041710,3.268613,3.169693,...,3.947261,2.117541,3.455042,1.585534,2.814844,3.112863,1.732506,0.187103,1.804562,1.817462
2,3.135633,0.572274,1.382039,1.417841,3.184246,3.683941,1.790872,3.770267,2.921054,2.561756,...,3.262166,2.537447,3.479069,1.272809,2.639281,3.102754,1.760821,0.165446,1.826111,1.669104
3,3.227951,0.624250,1.418705,1.472930,3.637292,3.738969,1.630933,4.132305,3.472255,3.280667,...,3.683004,2.258739,3.954709,1.447627,2.909541,3.282647,1.920817,0.165391,1.832204,1.640271
4,3.221246,0.534398,1.042870,1.043824,3.876853,3.337098,1.341508,3.877304,3.375667,3.084420,...,3.037269,1.985003,4.073300,1.561691,2.931189,3.190532,1.698327,0.166663,1.666597,1.458787
5,3.349985,0.638174,1.314823,1.371895,3.408788,3.763721,1.579509,4.151832,3.574903,3.662990,...,3.261184,2.321484,3.816144,1.340816,3.106211,3.625464,1.812592,0.340113,1.673094,2.427792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5077,2.461911,0.516051,1.651974,1.492821,2.964691,2.815689,2.100366,3.309192,2.907363,3.412535,...,2.105967,2.896624,2.643222,1.449590,2.412295,2.711798,1.631980,0.266952,2.133547,1.706961
5078,3.867898,0.770269,2.140225,2.139878,4.162214,3.432046,2.603764,4.353557,3.692170,3.018657,...,2.816953,3.765352,4.283115,2.048353,2.974550,4.070143,2.131491,0.328655,2.562149,2.807827
5079,3.380937,0.633906,1.486321,1.553899,3.453967,3.265186,1.784042,4.005635,3.366703,3.012599,...,2.590233,2.587712,3.599297,1.389360,2.896909,3.294317,2.077058,0.238177,1.900337,2.277170


In [135]:
ratings_pred_matrix.iloc[0,:].sort_values(ascending=False)

title
Besieged (a.k.a. L' Assedio) (1998)                 5.023612
Kiss Kiss Bang Bang (2005)                          5.007407
Social Network, The (2010)                          4.989622
Touching the Void (2003)                            4.978926
Arsenic and Old Lace (1944)                         4.940129
                                                      ...   
Batman vs. Two-Face (2017)                          0.016222
The Loyal 47 Ronin (1958)                          -0.022467
The Remarkable Mr. Pennypacker (1959)              -0.034786
Complete History of My Sexual Failures, A (2008)   -0.226782
Make Them Die Slowly (Cannibal Ferox) (1981)       -0.799967
Name: 1, Length: 17817, dtype: float64

In [32]:
# 사용자가 관람하지 않은 영화명 추출
unseen_list = get_unseen_movies(rating_matrix,1)


In [33]:
unseen_list

['"Great Performances" Cats (1998)',
 '#Horror (2015)',
 '$ (Dollars) (1971)',
 '$5 a Day (2008)',
 "'71 (2014)",
 "'Hellboy': The Seeds of Creation (2004)",
 "'R Xmas (2001)",
 "'Round Midnight (1986)",
 "'Salem's Lot (2004)",
 "'Til There Was You (1997)",
 "'Twas the Night Before Christmas (1974)",
 "'burbs, The (1989)",
 "'night Mother (1986)",
 '(500) Days of Summer (2009)',
 '(A)sexual (2011)',
 '(Dis)Honesty: The Truth About Lies (2015)',
 '(Untitled) (2009)',
 '*batteries not included (1987)',
 '+1 (2013)',
 '...And God Spoke (1993)',
 '...And Justice for All (1979)',
 '1 (2014)',
 '1 Journée (2007)',
 '1, 2, 3, Sun (Un, deuz, trois, soleil) (1993)',
 '1-900 (06) (1994)',
 '10 (1979)',
 '10 Cloverfield Lane (2016)',
 '10 Items or Less (2006)',
 '10 Questions for the Dalai Lama (2006)',
 '10 Rillington Place (1971)',
 '10 Things I Hate About You (1999)',
 '10 to Midnight (1983)',
 '10,000 BC (2008)',
 '10.000 Km (Long Distance) (2014)',
 '100 Feet (2008)',
 '100 Girls (2000)',
 '

In [129]:
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix,1, unseen_list,top_n=20)
recomm_movies

,pred_score
title,
Besieged (a.k.a. L' Assedio) (1998),5.023612
Kiss Kiss Bang Bang (2005),5.007407
"Social Network, The (2010)",4.989622
Touching the Void (2003),4.978926
Arsenic and Old Lace (1944),4.940129
WALL·E (2008),4.919814
"Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",4.903924
"Dirty Dozen, The (1967)",4.899222
"Ref, The (1994)",4.891753


----
---
---
---

In [120]:
ss = rating_movies.query('userId == 1')
ss

,userId,movieId,rating,title,genres
0,1,307,3.5,Three Colors: Blue (Trois couleurs: Bleu) (1993),[Drama]
167,1,481,3.5,Kalifornia (1993),"[Drama, Thriller]"
301,1,1091,1.5,Weekend at Bernie's (1989),[Comedy]
408,1,1257,4.5,Better Off Dead... (1985),"[Comedy, Romance]"
517,1,1449,4.5,Waiting for Guffman (1996),[Comedy]
642,1,1590,2.5,Event Horizon (1997),"[Horror, Sci-Fi, Thriller]"
798,1,1591,1.5,Spawn (1997),"[Action, Adventure, Sci-Fi, Thriller]"
905,1,2134,4.5,Weird Science (1985),"[Comedy, Fantasy, Sci-Fi]"
1024,1,2478,4.0,¡Three Amigos! (1986),"[Comedy, Western]"
1167,1,2840,3.0,Stigmata (1999),"[Drama, Thriller]"


In [130]:
aa = pd.merge(recomm_movies,movies, on='title',how='left')
aa['genres'] = aa['genres'].apply(lambda x: x.split('|'))
aa

,title,pred_score,movieId,genres
0,Besieged (a.k.a. L' Assedio) (1998),5.023612,2630,[Drama]
1,Kiss Kiss Bang Bang (2005),5.007407,38061,"[Comedy, Crime, Mystery, Thriller]"
2,"Social Network, The (2010)",4.989622,80463,[Drama]
3,Touching the Void (2003),4.978926,7256,"[Adventure, Documentary]"
4,Arsenic and Old Lace (1944),4.940129,1269,"[Comedy, Mystery, Thriller]"
5,WALL·E (2008),4.919814,60069,"[Adventure, Animation, Children, Romance, Sci-Fi]"
6,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",4.903924,4973,"[Comedy, Romance]"
7,"Dirty Dozen, The (1967)",4.899222,2944,"[Action, Drama, War]"
8,"Ref, The (1994)",4.891753,514,[Comedy]
9,Short Term 12 (2013),4.891584,104944,[Drama]


---
---
---
### 영화 데이터 약간의 전처리

이걸 멜론데이터에 사용하면 될 듯

In [122]:
# rating_movies['genres'] = rating_movies['genres'].apply(lambda x: x.split('|'))
rating_movies

,userId,movieId,rating,title,genres
0,1,307,3.5,Three Colors: Blue (Trois couleurs: Bleu) (1993),[Drama]
1,6,307,4.0,Three Colors: Blue (Trois couleurs: Bleu) (1993),[Drama]
2,56,307,4.0,Three Colors: Blue (Trois couleurs: Bleu) (1993),[Drama]
3,71,307,5.0,Three Colors: Blue (Trois couleurs: Bleu) (1993),[Drama]
4,84,307,3.0,Three Colors: Blue (Trois couleurs: Bleu) (1993),[Drama]
...,...,...,...,...,...
499995,5069,80454,3.0,Princess (Prinsessa) (2010),[Drama]
499996,5069,80584,3.0,"Patrik Age 1.5 (Patrik 1,5) (2008)","[Comedy, Drama, Romance]"
499997,5069,83760,1.5,V2: Dead Angel (Vares 2 - Jäätynyt Enkeli) (2007),"[Action, Comedy, Crime]"
499998,5073,439,3.0,Dangerous Game (1993),[Drama]


In [123]:
ss

,userId,movieId,rating,title,genres
0,1,307,3.5,Three Colors: Blue (Trois couleurs: Bleu) (1993),[Drama]
167,1,481,3.5,Kalifornia (1993),"[Drama, Thriller]"
301,1,1091,1.5,Weekend at Bernie's (1989),[Comedy]
408,1,1257,4.5,Better Off Dead... (1985),"[Comedy, Romance]"
517,1,1449,4.5,Waiting for Guffman (1996),[Comedy]
642,1,1590,2.5,Event Horizon (1997),"[Horror, Sci-Fi, Thriller]"
798,1,1591,1.5,Spawn (1997),"[Action, Adventure, Sci-Fi, Thriller]"
905,1,2134,4.5,Weird Science (1985),"[Comedy, Fantasy, Sci-Fi]"
1024,1,2478,4.0,¡Three Amigos! (1986),"[Comedy, Western]"
1167,1,2840,3.0,Stigmata (1999),"[Drama, Thriller]"


In [124]:
ss['genres'].value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[Drama, Thriller]                            2
[Drama]                                      2
[Horror, Sci-Fi, Thriller]                   2
[Comedy]                                     2
[Action, Adventure, Sci-Fi, Thriller]        1
[Action, Drama]                              1
[Comedy, Crime, Drama, Romance, Thriller]    1
[Comedy, Fantasy, Sci-Fi]                    1
[Action, Crime, Sci-Fi, Thriller]            1
[Action, Sci-Fi]                             1
[Comedy, Romance]                            1
[Comedy, Western]                            1
Name: genres, dtype: int64

In [138]:
# 리스트로 묶여 있는 장르를 다 풀어주는 거

# 이렇게 진행하면 장르별 추천이 더 수월하지 않을까라는 생각

s = ss.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'

gen_md = ss.drop('genres', axis=1).join(s)
gen_md

,userId,movieId,rating,title,genre
0,1,307,3.5,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
167,1,481,3.5,Kalifornia (1993),Drama
167,1,481,3.5,Kalifornia (1993),Thriller
301,1,1091,1.5,Weekend at Bernie's (1989),Comedy
408,1,1257,4.5,Better Off Dead... (1985),Comedy
408,1,1257,4.5,Better Off Dead... (1985),Romance
517,1,1449,4.5,Waiting for Guffman (1996),Comedy
642,1,1590,2.5,Event Horizon (1997),Horror
642,1,1590,2.5,Event Horizon (1997),Sci-Fi
642,1,1590,2.5,Event Horizon (1997),Thriller


0                                         [Drama]
167                             [Drama, Thriller]
301                                      [Comedy]
408                             [Comedy, Romance]
517                                      [Comedy]
642                    [Horror, Sci-Fi, Thriller]
798         [Action, Adventure, Sci-Fi, Thriller]
905                     [Comedy, Fantasy, Sci-Fi]
1024                            [Comedy, Western]
1167                            [Drama, Thriller]
1265            [Action, Crime, Sci-Fi, Thriller]
1355                              [Action, Drama]
1488                                      [Drama]
1630                             [Action, Sci-Fi]
1781                   [Horror, Sci-Fi, Thriller]
1921    [Comedy, Crime, Drama, Romance, Thriller]
Name: genres, dtype: object